# Face Recognition Attendance Project

#### Import Necessary Dorectories

In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=f37e692f2cfa7e2f6bd4e7a458a1341270ef15aab6007093da86b26531d0b790
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

#### Get Image Names as classnames for classification

In [ ]:
#path = '/home/punnoose1997/Documents/Ophir Projects/Face Attendance Project/Face-Recognition-Attendance-Projects-main/Training_images'
path = '/content/drive/MyDrive/Face Recognition Attendance Project/Training_images'
images = []
classNames = []
myList = os.listdir(path)
print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Christo Baby.jpeg', 'Sudhanshu.jpg', 'Krish Naik.jpg', 'prince.jpg']
['Christo Baby', 'Sudhanshu', 'Krish Naik', 'prince']


#### Function to Find face encodings from sample images

In [ ]:
def findEncodings(images):
    encodeList = []
    
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

#### Function to Mark attendance of identified person on Attendance.csv file

In [ ]:
def markAttendance(name):
    #with open('/home/punnoose1997/Documents/Ophir Projects/Face Attendance Project/Face-Recognition-Attendance-Projects-main/Attendance.csv', 'r+') as f:
    with open('/content/drive/MyDrive/Face Recognition Attendance Project/Attendance.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

In [ ]:
def markUnknown(name):
    #with open('/home/punnoose1997/Documents/Ophir Projects/Face Attendance Project/Face-Recognition-Attendance-Projects-main/UnknownList.csv', 'r+') as f:
    with open('/content/drive/MyDrive/Face Recognition Attendance Project/UnknownList.csv', 'r+') as f:
        myDataList = f.readlines()


        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')

In [ ]:
!pip3 install twilio

     |████████████████████████████████| 1.4 MB 4.1 MB/s 


In [ ]:
import os
from twilio.rest import Client

def sendAlertTwilio(name):
    #This function sends alerts to specified Phone numbers
    account_sid = os.environ['AC4ae98176d5ff9b9eb257ea8c2503991b']
    auth_token = os.environ['b0c60ffc5b10ac8cd4fed2051ce37e65']
    target_number = '+918547999613'
    source_number = '+91 89438 42248'
    client = Client(account_sid, auth_token)
    message = client.messages.create(body=name+' has entered frame.', from_=source_number, to=target_number)
    print(message.sid)
    

#### Find Encodings and print them.
#### Capture system webcam input

In [ ]:
encodeListKnown = findEncodings(images)
print('Encoding Complete')

cap = cv2.VideoCapture(0)

Encoding Complete


#### Process captured images, find face locations, encodings, compare faces, and add them to attendance

In [ ]:
while True:
    success, img = cap.read()
# img = captureScreen()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
# print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
# print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
        else:
            name = "Unknown Person"
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markUnknown(name)
            sendAlertTwilio(name)
            

    cv2.imshow('Webcam', img)
    cv2.waitKey(1)

error: OpenCV(4.4.0) /tmp/pip-req-build-v1rk7obu/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
